In [1]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm
    

c:\Users\hodooai\anaconda3\envs\mnist\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# CUDA(GPU) 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 데이터 전처리 및 데이터 로더 생성
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # transforms.Grayscale(),
])

train_imgfolder = ImageFolder(root='C:/Users/hodooai/Downloads/2021_Huons_Train_Line2_Inspected_v2/train', transform=preprocess)
val_imgfolder = ImageFolder(root='C:/Users/hodooai/Downloads/2021_Huons_Train_Line2_Inspected_v2/valid', transform=preprocess)
test_imgfolder = ImageFolder(root='C:/Users/hodooai/Downloads/2021_Huons_Train_Line2_Inspected_v2/test', transform=preprocess)

train_loader = DataLoader(train_imgfolder,batch_size=64,shuffle=True)
val_loader = DataLoader(val_imgfolder,batch_size=32,shuffle=True)
test_loader = DataLoader(test_imgfolder,batch_size=32,shuffle=True)

In [4]:
#Root
# fully connected network network
# Convolutional Neural Network

class CNN_Model(torch.nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, 3, 1, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 16, 2, 1, padding=1)
        self.fc1 = torch.nn.Linear(50176, 4)

    def forward(self, x):
        # x = x.view(-1, 3, 224, 224)
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.max_pool2d(x, 2)
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(x, 2)
        x = torch.nn.Flatten()(x)
        x = self.fc1(x)
        return x

In [ ]:
# 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(224*224, 224*224//16)  # 784 = 28(width) x 28(height)
        self.fc2 = nn.Linear(224*224//16, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 4)

    def forward(self, x):
        #
        x = x.view(-1, 224*224)#다른차원의 크기를 자동으로 조정하는 함수//  # m차원 값을 n차원으로 변경 -1 = 아무거나, 784 = 28 x 28 #  PyTorch에서 텐서의 형태를 변경(reshape)하기 위해 사용되는 메서드입니다

        
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.relu(self.fc4(x))
        x = nn.functional.relu(self.fc5(x))
        x = nn.functional.dropout(x, training=self.training)
        x = self.fc6(x)
        return x

In [5]:
from torchvision.models import vgg11_bn as CNN_Model

In [7]:


# 멀티 클래스 - 결과 값은 한개 - 현재 테스크 합이 1 - 현재 테스크
# 멀티 라벨 - 결과 값이 여러개  합이 1 이상 일 수 있음
# 모델 생성 및 CUDA로 전송
m3 = CNN_Model().to(device)

# Loss 함수 및 Optimizer 설정
criterion = nn.CrossEntropyLoss() # 여기서 확률 1로 변경해줌 떄때로
optimizer = optim.Adam(m3.parameters(), lr=5e-5)

# 학습 루프
for epoch in range(1):
    start = time.time()
    total_loss = 0

    for image, label in train_loader:
        image, label = image.to(device=device), label.to(device=device)
        # print(xb.shape, yb.shape)
        pred = m3(image)
        loss : torch.Tensor = criterion(pred, label)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        print(f"{time.time() - start} train_loss: {loss}")
    val_acc = 0
    val_loss = 0
    with torch.no_grad():        
        for image, labels in val_loader:
            image, label = image.to(device=device), label.to(device=device)
            pred = m3(image)
            # print(pred.data.max(1)[1], labels.data)
            acc = pred.data.max(1)[1].eq(labels.data).sum().item() / len(image) * 100
            loss = criterion(pred, labels)
            val_acc += acc
            val_loss+= loss
    val_acc /= len(val_loader)
    val_loss /= len(val_loader)
    print(f"{time.time() - start} sec - loss: {val_loss} / acc: {val_acc}")

7.064342737197876 train_loss: 7.185851573944092
9.90731143951416 train_loss: 2.091719627380371
12.707985401153564 train_loss: 1.289452075958252
15.526219367980957 train_loss: 1.1063954830169678
18.429199695587158 train_loss: 1.8473533391952515
21.21617102622986 train_loss: 0.9378305673599243
24.030972480773926 train_loss: 1.4273343086242676
26.843449115753174 train_loss: 2.2281882762908936
29.715237617492676 train_loss: 1.231534719467163
32.563363790512085 train_loss: 1.8437926769256592
35.379690170288086 train_loss: 1.3182889223098755
38.22990846633911 train_loss: 1.0407633781433105
41.13417863845825 train_loss: 1.2058112621307373
43.921844244003296 train_loss: 1.4218957424163818
46.75265383720398 train_loss: 1.0268017053604126
49.6490592956543 train_loss: 0.9913254380226135
52.50289011001587 train_loss: 1.4277052879333496
55.42893171310425 train_loss: 2.2016282081604004
58.35615921020508 train_loss: 0.8102055191993713
61.268059730529785 train_loss: 1.1612221002578735
64.1852071285247

In [8]:
#테스트 코드
test_acc = 0
test_loss = 0
with torch.no_grad():
    for image, label in test_loader:
        image = image.to(device=device)
        label = label.to(device=device)

        pred = m3(image)
        # print(pred.data.max(1)[1], labels.data)
        acc = pred.data.max(1)[1].eq(label.data).sum().item() / len(image) * 100
        loss = criterion(pred, label)
        test_acc += acc
        test_loss+= loss
print(f"{time.time() - start} sec - loss: {val_loss} / acc: {val_acc}")

95.27933526039124 sec - loss: 1.5686782598495483 / acc: 28.125


In [14]:
#모델 저장
torch.save(m3.state_dict(), "D:/test/a.pth")

In [23]:
#모델 불러오기
model = CNN_Model()  # 모델을 생성하거나 초기화합니다.
model.load_state_dict(torch.load('D:/test/a.pth'))
model.eval()  # 모델을 평가 모드로 설정합니다.
model.to(device=device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [30]:
#테스트 코드
test_acc = 0
test_loss = 0
with torch.no_grad():
    for image, label in test_loader:
        image = image.to(device=device)
        label = label.to(device=device)

        pred = model(image)
        # print(pred.data.max(1)[1], labels.data)
        acc = pred.data.max(1)[1].eq(label.data).sum().item() / len(image) * 100
        loss = criterion(pred, label)
        test_acc += acc
        test_loss+= loss
print(f"{time.time() - start} sec - loss: {val_loss} / acc: {val_acc}")

3002.937858104706 sec - loss: 1.5686782598495483 / acc: 28.125
